In [2]:
import numpy as np
from losses import *

In [3]:
BATCH = 3
SEQ_LEN = 1024
N_CLASSES = 4

# random probabilities
x = np.random.rand(BATCH, SEQ_LEN, N_CLASSES)

# sparsely encoded (random) labels
y = np.random.randint(-1, N_CLASSES, (BATCH, SEQ_LEN))

In [5]:
l = MaskedTemporalCategoricalCrossEntropy(from_logits=False, sparse=True, n_classes=2)


I0000 00:00:1733773396.691563   45434 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9887 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [11]:
y.shape

(3, 1024)

In [13]:
x.shape

(3, 1024, 4)